In [1]:
import warnings
warnings.simplefilter("ignore", FutureWarning)


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
from pathlib import Path
import time

# LightGBM
from lightgbm import LGBMClassifier

# sklearn
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import (
    train_test_split, KFold, cross_validate, cross_val_score,
    RandomizedSearchCV, StratifiedKFold
)

from sklearn.metrics import (
    classification_report, confusion_matrix,
    accuracy_score, roc_auc_score
)

from sklearn.base import BaseEstimator, TransformerMixin, clone
from scipy.stats import randint, uniform, loguniform

# Importações locais
from setup_notebook import setup_path
setup_path()

from src.model_utils import *
from src.preprocess_utils_diab3 import * #(NOVO atualizações)

print("\n#Processo iniciado em:", time.strftime("%H:%M:%S"))
start_inicial = time.time()


#Processo iniciado em: 21:36:00


## 1. Load Data & Pipeline

In [2]:
BASE = Path.cwd().parent

PP3 = joblib.load(BASE/'src'/'preprocess_diabetes_v3.joblib')['preprocessador']

DATA_DIR = BASE/"data"/"raw"
X_train = pd.read_csv(DATA_DIR/"X_train_raw.csv")
X_test  = pd.read_csv(DATA_DIR/"X_test_raw.csv")
y_train = pd.read_csv(DATA_DIR/"y_train_raw.csv").values.ravel()
y_test  = pd.read_csv(DATA_DIR/"y_test_raw.csv").values.ravel()
mtd_scoring='roc_auc'

## 2.Baseline

In [3]:
def  best_threshold(y_test,yprob):
    # Threshold search
    thresholds = np.linspace(0.3, 0.7, 41)
    best_val = 0.5
    max_acc = 0
    for t in thresholds:
        acc = accuracy_score(y_test, yprob > t)
        if acc > max_acc:
            max_acc = acc
            best_val = t
    return best_val,max_acc

def print_hiper(search_best_params):
    
    print("🎯 Melhores Hiperparâmetros")
    print("-" * 50)
    for param, value in search_best_params.items():
        param_name = param.replace('model__', '').replace('_', ' ').title()
        print(f"• {param_name:<25} : {value}")
    print("-" * 50)

#=====================================================================
# model Baseline
print("\n#Processo iniciado em:", time.strftime("%H:%M:%S"))
model_base = LGBMClassifier(objective="binary",n_jobs=-1,random_state=42,verbose=-1,
                            force_row_wise=True)

pipe_base  = pipe_models(model_base, PP3)

cv_s = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

pipe_base.fit(X_train, y_train)
y_probs0 = pipe_base.predict_proba(X_test)[:, 1]

threshold_0,max_acc_0=best_threshold(y_test,y_probs0)
auc_score_0 = roc_auc_score(y_test, y_probs0)

print(f"{'='*70}")
print(f"🎯 Melhor Threshold         : {threshold_0:.3f}")
print(f"📈 Melhor Acurácia de Teste : {max_acc_0:.4f}")
print(f"⭐ AUC Score                : {auc_score_0:.4f}")
print(f"{'='*70}")
print_hiper(model_base.get_params())

print("\n#Processo finalizado em:", time.strftime("%H:%M:%S"))


#Processo iniciado em: 21:36:01
🎯 Melhor Threshold         : 0.500
📈 Melhor Acurácia de Teste : 0.6806
⭐ AUC Score                : 0.7210
🎯 Melhores Hiperparâmetros
--------------------------------------------------
• Boosting Type             : gbdt
• Class Weight              : None
• Colsample Bytree          : 1.0
• Importance Type           : split
• Learning Rate             : 0.1
• Max Depth                 : -1
• Min Child Samples         : 20
• Min Child Weight          : 0.001
• Min Split Gain            : 0.0
• N Estimators              : 100
• N Jobs                    : -1
• Num Leaves                : 31
• Objective                 : binary
• Random State              : 42
• Reg Alpha                 : 0.0
• Reg Lambda                : 0.0
• Subsample                 : 1.0
• Subsample For Bin         : 200000
• Subsample Freq            : 0
• Verbose                   : -1
• Force Row Wise            : True
--------------------------------------------------

#Processo f

## 3.Buscas por hiperparamentros


In [4]:
# exploratorio
print("# Processo Iniciado em:", time.strftime("%H:%M:%S"))
param_dist_1 = {
    # Boosting
    'model__n_estimators': randint(200, 3000),
    'model__learning_rate': loguniform(0.005, 0.3),

    # Estrutura
    'model__max_depth': randint(3, 10),
    'model__num_leaves': randint(40, 200),

    # Regularização
    'model__min_child_samples': randint(10, 100),
    'model__subsample': uniform(0.6, 0.4),
    'model__colsample_bytree': uniform(0.1, 0.9),
    'model__reg_alpha': uniform(0, 5),
    'model__reg_lambda': [0, 0.5, 1, 3, 5, 10]
}

cv_s = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
numero_itera = 50

search_1 = RandomizedSearchCV(
    pipe_base,
    param_dist_1,
    n_iter=numero_itera,
    cv=cv_s,
    scoring=mtd_scoring,
    random_state=42,
    verbose=1
)

start = time.time()
search_1.fit(X_train, y_train)
end = time.time()

best_1 = search_1.best_estimator_

y_probs1 = best_1.predict_proba(X_test)[:, 1]

threshold_1,max_acc_1=best_threshold(y_test,y_probs1)
auc_score_1 = roc_auc_score(y_test, y_probs1)

print(f"{'='*70}")
print(f"🎯 Melhor Threshold         : {threshold_1:.3f}")
print(f"📈 Melhor Acurácia de Teste : {max_acc_1:.4f}")
print(f"⭐ AUC Score                : {auc_score_1:.4f}")
print(f"{'='*70}")
print_hiper(search_1.best_params_)

print(f"Tempo total: {end-start:.2f} segundos")
print(f"Tempo por iteração: {(end-start)/numero_itera:.2f} segundos")
print("\n#Processo finalizado em:", time.strftime("%H:%M:%S"))


# Processo Iniciado em: 21:36:05
Fitting 3 folds for each of 50 candidates, totalling 150 fits
🎯 Melhor Threshold         : 0.500
📈 Melhor Acurácia de Teste : 0.6857
⭐ AUC Score                : 0.7282
🎯 Melhores Hiperparâmetros
--------------------------------------------------
• Colsample Bytree          : 0.10496990541124217
• Learning Rate             : 0.1409236110165583
• Max Depth                 : 3
• Min Child Samples         : 17
• N Estimators              : 1898
• Num Leaves                : 72
• Reg Alpha                 : 3.029799873905057
• Reg Lambda                : 0.5
• Subsample                 : 0.7433862914177091
--------------------------------------------------
Tempo total: 4266.26 segundos
Tempo por iteração: 85.33 segundos

#Processo finalizado em: 22:47:16


In [5]:
print("#Processo Iniciado em:", time.strftime("%H:%M:%S"))
def generate_refined_grid(best_params):
    """
    Gera um grid refinado automaticamente baseado nos melhores parâmetros
    do Random Search exploratório (LightGBM).
    """
    # ---------------------------
    # 1. n_estimators
    # ---------------------------
    n_best = best_params['model__n_estimators']
    n_min  = int(n_best * 0.8)
    n_max  = int(n_best * 1.2)
    # ---------------------------
    # 2. learning_rate
    # ---------------------------
    lr_best = best_params['model__learning_rate']
    lr_st   = lr_best * 0.7
    lr_end  = lr_best * 1.3
    # ---------------------------
    # 3. max_depth
    # ---------------------------
    depth_best = best_params['model__max_depth']
    depth_list = [max(1, depth_best - 1),
                  depth_best,depth_best + 1 ]
    # ---------------------------
    # 4. num_leaves
    # ---------------------------
    nl_best = best_params['model__num_leaves']
    nl_min  = max(8, int(nl_best * 0.7))
    nl_max  = int(nl_best * 1.3)

    # ---------------------------
    # 5. min_child_samples
    # ---------------------------
    mcs_best = best_params['model__min_child_samples']
    mcs_min  = max(5, int(mcs_best * 0.7))
    mcs_max  = int(mcs_best * 1.3)
    # ---------------------------
    # 6. subsample
    # ---------------------------
    ss_best = best_params['model__subsample']
    ss_st   = ss_best * 0.8
    ss_end  = min(1.0, ss_best + 0.2)
    # ---------------------------
    # 7. colsample_bytree
    # ---------------------------
    cb_best = best_params['model__colsample_bytree']
    cb_st   = cb_best * 0.8
    cb_end  = min(1.0, cb_best + 0.2)
    # ---------------------------
    # 8. regularização
    # ---------------------------
    alpha_best  = best_params['model__reg_alpha']
    lambda_best = best_params['model__reg_lambda']

    alpha_st = alpha_best * 0.7
    alpha_end = alpha_best + 0.5

    lambda_st = lambda_best * 0.7
    lambda_end = lambda_best + 0.5

    # ---------------------------
    # PRINTS DIAGNÓSTICOS
    # ---------------------------
    print(f"{'='*30} NOVO GRID DE REFINAMENTO (LightGBM) {'='*30}")
    print(f"🔹 n_estimators        : {n_min}  - {n_max}")
    print(f"🔹 learning_rate       : {lr_st:.5f} - {lr_end:.5f}")
    print(f"🔹 max_depth           : {depth_list}")
    print(f"🔹 num_leaves          : {nl_min} - {nl_max}")
    print(f"🔹 min_child_samples   : {mcs_min} - {mcs_max}")
    print(f"🔹 subsample           : {ss_st:.3f} - {ss_end:.3f}")
    print(f"🔹 colsample_bytree    : {cb_st:.3f} - {cb_end:.3f}")
    print(f"🔹 reg_alpha           : {alpha_st:.4f} - {alpha_end:.4f}")
    print(f"🔹 reg_lambda          : {lambda_st:.4f} - {lambda_end:.4f}")
    print(f"{'='*86}\n")

    # ---------------------------
    # GRID FINAL
    # ---------------------------
    refined_grid = {
        'model__n_estimators': randint(n_min, n_max + 1),
        'model__learning_rate': uniform(lr_st, lr_end - lr_st),
        'model__max_depth': depth_list,
        'model__num_leaves': randint(nl_min, nl_max + 1),
        'model__min_child_samples': randint(mcs_min, mcs_max + 1),
        'model__subsample': uniform(ss_st, ss_end - ss_st),
        'model__colsample_bytree': uniform(cb_st, cb_end - cb_st),
        'model__reg_alpha': uniform(alpha_st, alpha_end - alpha_st),
        'model__reg_lambda': uniform(lambda_st, lambda_end - lambda_st)
    }

    return refined_grid


# Aplicação automática
param_dist_2 = generate_refined_grid(search_1.best_params_)


numero_itera = 30
search_2 = RandomizedSearchCV(
    pipe_base, param_dist_2,
    n_iter=numero_itera, cv=cv_s, 
    scoring=mtd_scoring,
    random_state=42, verbose=1
)

start = time.time()
search_2.fit(X_train, y_train)
end = time.time()

best_2 = search_2.best_estimator_
y_probs2 = best_2.predict_proba(X_test)[:, 1]

threshold_2,max_acc_2=best_threshold(y_test,y_probs2)
auc_score_2 = roc_auc_score(y_test, y_probs2)

print(f"{'='*70}")
print(f"🎯 Melhor Threshold         : {threshold_2:.3f}")
print(f"📈 Melhor Acurácia de Teste : {max_acc_2:.4f}")
print(f"⭐ AUC Score                : {auc_score_2:.4f}")
print(f"{'='*70}")
print_hiper(search_2.best_params_)

print(f"Tempo total: {end-start:.2f} segundos")
print(f"Tempo por iteração: {(end-start)/numero_itera:.2f} segundos")
print("\n#Processo finalizado em:", time.strftime("%H:%M:%S"))


#Processo Iniciado em: 22:47:16
============================== NOVO GRID DE REFINAMENTO (LightGBM) ==============================
🔹 n_estimators        : 1518  - 2277
🔹 learning_rate       : 0.09865 - 0.18320
🔹 max_depth           : [2, 3, 4]
🔹 num_leaves          : 50 - 93
🔹 min_child_samples   : 11 - 22
🔹 subsample           : 0.595 - 0.943
🔹 colsample_bytree    : 0.084 - 0.305
🔹 reg_alpha           : 2.1209 - 3.5298
🔹 reg_lambda          : 0.3500 - 1.0000

Fitting 3 folds for each of 30 candidates, totalling 90 fits
🎯 Melhor Threshold         : 0.500
📈 Melhor Acurácia de Teste : 0.6859
⭐ AUC Score                : 0.7285
🎯 Melhores Hiperparâmetros
--------------------------------------------------
• Colsample Bytree          : 0.148009251432773
• Learning Rate             : 0.11227845929513408
• Max Depth                 : 3
• Min Child Samples         : 20
• N Estimators              : 2074
• Num Leaves                : 55
• Reg Alpha                 : 2.764089712199721
• Reg Lambd

In [6]:
end_final = time.time()
print(f"Tempo final: {(end_final-start_inicial)/60:.2f} min")


Tempo final: 98.31 min


In [7]:
#Salvar Hiperparametros joblib
joblib.dump(search_1.best_estimator_, 'modelo_LGBM2_final_randsearch.'+mtd_scoring+'_v3.joblib')
joblib.dump(search_2.best_estimator_, 'modelo_LGBM2_final_refine.'+mtd_scoring+'_v3.joblib')

['modelo_LGBM2_final_refine.roc_auc_v3.joblib']